In this notebook, we will walk you through the steps to launch a [Cloud Build](https://cloud.google.com/build) job to:

* Clone the repository where the code is hosted.
* Compile a TFX (TensorFlow Extended) pipeline.
* Build a Docker image with custom TFX components.
* Submit the pipeline to Vertex AI for execution. 

Below you can find a pictorial overview of the overall workflow:

![](https://i.ibb.co/bzS8vzZ/first-workflow.png)

**Note** that we won't be covering the part related to GitHub Actions. For that, we refer the interested readers to our blog post instead.

For the best experience, we suggest running this notebook from [Vertex AI Notebooks](https://cloud.google.com/vertex-ai/docs/general/notebooks).


## Step 0

We first need to have a Docker image ready which we could use with Cloud Build to perform the mentioned steps. We first write a `Dockerfile` which is going to be used to build and push the image to Google Container Registry (GCR).

In [ ]:
%%writefile Dockerfile

FROM tensorflow/tfx:1.2.0
RUN pip install kfp==1.7.1 pytest

In [ ]:
TFX_IMAGE_URI = "gcr.io/{gcp-project}/{name}"
!gcloud builds submit --tag $TFX_IMAGE_URI . --timeout=15m --machine-type=e2-highcpu-8

This may take some time. 

## Step 1

With the initial Docker image ready, we are good to proceed to the next steps. Here we will be manually triggering a build and for that we need to define a couple of variables. But note that this project is configured with a GitHub Action workflow that monitors code changes made to a certain directory and triggers builds automatically based on that.

In [ ]:
# You'd need to change these values accordingly.

SUBSTITUTIONS= """
   _PROJECT=gcp-ml-172005,
   _REGION=us-central1,
   _REPO_URL=https://github.com/deep-diver/Model-Training-as-a-CI-CD-System,
   _BRANCH=main,
   _PIPELINE_NAME=tfx-pipeline
"""

## Step 2

Clone the repository to get the build specification. You can know more about specification files needed for Cloud Build from [here](https://cloud.google.com/build/docs/build-config-file-schema).

In [ ]:
!git clone https://github.com/deep-diver/Model-Training-as-a-CI-CD-System

## Step 3

Now, we can submit to Cloud Build.

In [ ]:
%cd Model-Training-as-a-CI-CD-System
!gcloud builds submit --no-source --timeout=60m \
    --config build/pipeline-deployment.yaml \
    --substitutions $SUBSTITUTIONS \
    --machine-type=n1-highcpu-8